# dim_forneceedores

Esse notebook é responsável por reunir todas as informações úteis de cada fornecedor do e-commerce


## Configurações Iniciais

### Importando as Bibliotecas


In [0]:
from pyspark.sql import functions as F

### Carregando os Dados

In [0]:
df_sellers = (
    spark.read.csv(
        "/Volumes/workspace/default/e_commerce/olist_sellers_dataset.csv",
        header=True,
        inferSchema=True
    )
    .drop("seller_city", "seller_state")
)

## Salvamento dos Dados

In [0]:
%sql
CREATE TABLE IF NOT EXISTS ifrs_dev_ecommerce.gold.dim_fornecedores (
    seller_id STRING PRIMARY KEY COMMENT "ID do fornecedor",
    seller_zip_code_prefix INT COMMENT "Prefixo do CEP do fornecedor",
    seller_name STRING COMMENT "Nome do fornecedor",
    CONSTRAINT fk_geolocalizacao_sellers FOREIGN KEY (seller_zip_code_prefix)
        REFERENCES ifrs_dev_ecommerce.gold.dim_geolocalizacao(geolocation_zip_code_prefix)
)
USING DELTA
CLUSTER BY (seller_id, seller_zip_code_prefix)
TBLPROPERTIES ("quality"="gold", "layer"="data-engineering")
COMMENT "Dimensão de fornecedores contendo informações de identificação e localização.";

-- Adicionando tags à tabela
ALTER TABLE ifrs_dev_ecommerce.gold.dim_fornecedores SET TAGS ("quality"="gold", "layer"="data-engineering");

In [0]:
(
    df_sellers
    .write.mode("overwrite")
    .format("delta")
    .saveAsTable("ifrs_dev_ecommerce.gold.dim_fornecedores")
)